<a href="https://colab.research.google.com/github/ishika-244/ishika-244/blob/main/Data_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Libraries**

In [7]:
!pwd

/content


In [8]:
pip install yfinance pandas numpy schedule

**importing libraries**

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import schedule
import os
import time

**Loading Data**

In [10]:
data = pd.read_csv('cleaned_stock_data.csv')

**Fetching and Cleaning Live TCS Data**

In [11]:
def fetch_and_clean_tcs():
  try:
    # Define Stock Symbol
    stock_symbol = "TCS.NS"
    stock_data = yf.Ticker(stock_symbol)

    # Fetch latest stock data for today
    df = stock_data.history(period="1d")

    # check if data was successfully fetched
    if stock_data.empty:
      print("No data fetched for TCS.NS")
      return

    # Format the data
    stock_data.insert(0 , "Date" , df.index.date)
    stock_data = stock_data[[('Date', 'Open', 'High', 'Low', 'Close', 'Volume','Dividends','sma_50','sma_200','ema_50','RSI','ATR','MACD','nifty_close','crude_oil','usd_inr','daily_range','Close_Smoothed')]]


    # Data Cleaning abd Validation Steps
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

    invalid_prices = stock_data[(stock_data['High']<stock_data['Low']) |
                         (stock_data['Open']>stock_data['High']) | (stock_data['Open'] < stock_data['Low']) |
                         (stock_data['Close']>stock_data['High']) | (stock_data['Close'] < stock_data['Low'])]
     # Validate Price Relationship
    if not invalid_prices.empty:
           print("Invalid Price Relationship Found:")
           print(invalid_prices)
    else:
        print("No Invalid Price Relationship Found.")


    # Check for zero volumes
    invalid_volumes = stock_data[stock_data['Volume'] <=0]
    if not invalid_volumes.empty:
     print("Invalid Volumes Found , replacing 0 with mean volume...:")
     stock_data['Volume'] = stock_data['Volume'].replace(0,stock_data['Volume'].mean())
     print("Invalid Volumes Handled")
    else:
      print("No Invalid Volumes Found.")


    # Check for Negative Volumes
    stock_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'sma_50', 'sma_200',
                 'ema_50', 'RSI', 'MACD', 'ATR', 'nifty_close', 'usd_inr', 'crude_oil']
    negative_values = stock_data[stock_columns] < 0
    if negative_values.any().any():
     print("Negative Values Found , replaced with Absolute Values")
     stock_data[stock_columns] = stock_data[stock_columns].abs()
     print("Negative Values Handled")
    else:
     print("No Negative Values Found.")


    # Validate SMA Calculation
    stock_data['sma_50_check'] =stock_data['Close'].rolling(window=50,adjust=False).mean()
    sma_mismatch = stock_data[abs(stock_data['sma_50'] - stock_data['sma_50_check']) > 1e-2]  # Allowing small rounding error
    if not sma_mismatch.empty:
     print("SMA Mismatch Found:")
     stock_data['sma_50'] = stock_data['Close'].rolling(window=50, adjust=False).mean()
     print(sma_mismatch)
    else:
     print("No SMA Mismatch Found.")


    # Validate EMA Calculation
    stock_data['ema_50_check'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
    ema_mismatch = stock_data[abs(stock_data['ema_50'] - stock_data['ema_50_check']) > 1e-2]
    # Print results
    if not ema_mismatch.empty:
     print("⚠️ EMA Mismatch Found and handled")
     stock_data['ema_50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
    # print(ema_mismatch[['Date', 'ema_50', 'ema_50_check']])
    else:
     print("✅ No EMA Mismatch Found.")


    # Calculate RSI (Relative Strength Index)
    delta = stock_data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    stock_data['RSI_check'] = 100 - (100 / (1 + rs))
    # Check for mismatches
    rsi_mismatch = stock_data[abs(stock_data['RSI'] - stock_data['RSI_check']) > 1e-2]
    # Print results
    if not rsi_mismatch.empty:
     print("⚠️ RSI Mismatch Found and handled")
     stock_data['RSI'] = 100 - (100 / (1 + (stock_data['Close'].diff().where(stock_data['Close'].diff() > 0, 0).rolling(window=14).mean() / (-stock_data['Close'].diff().where(stock_data['Close'].diff() < 0, 0).rolling(window=14).mean()))))
    # print(rsi_mismatch[['Date', 'RSI', 'RSI_check']])
    else:
     print("✅ No RSI Mismatch Found.")


    # Calculate MACD (Moving Average Convergence Divergence)
    stock_data['macd_line_check'] = stock_data['Close'].ewm(span=12, adjust=False).mean() - stock_data['Close'].ewm(span=26, adjust=False).mean()
    # Check for mismatches
    macd_mismatch = stock_data[abs(stock_data['MACD'] - stock_data['macd_line_check']) >1e-2]
    # Print results
    if not macd_mismatch.empty:
     print("⚠️ MACD Mismatch Found and handled")
     stock_data['MACD'] = stock_data['Close'].ewm(span=12, adjust=False).mean() - stock_data['Close'].ewm(span=26, adjust=False).mean()
    # print(macd_mismatch[['Date', 'MACD', 'macd_line_check']])
    else:
     print("✅ No MACD Mismatch Found.")



    # Calculate True Range (TR)
    stock_data['TR'] = np.maximum(
    stock_data['High'] - stock_data['Low'],
    np.maximum(abs(stock_data['High'] - stock_data['Close'].shift(1)),
               abs(stock_data['Low'] - stock_data['Close'].shift(1)))
                )
   # Calculate ATR (Average True Range)
    stock_data['ATR_check'] = stock_data['TR'].rolling(window=14).mean()
   # Check for mismatches
    atr_mismatch = stock_data[abs(stock_data['ATR'] - stock_data['ATR_check']) > 1e-2]
    # Print results
    if not atr_mismatch.empty:
     print("⚠️ ATR Mismatch Found and handled:")
     stock_data['ATR'] = pd.concat([(stock_data['High'] - stock_data['Low']), abs(stock_data['High'] - stock_data['Close'].shift(1)), abs(stock_data['Low'] - stock_data['Close'].shift(1))], axis=1).max(axis=1).rolling(window=14).mean()
     # print(atr_mismatch[['Date', 'ATR', 'ATR_check']])
    else:
     print("✅ No ATR Mismatch Found.")


     # Price Spikes
    stock_data['daily_range'] = stock_data['High'] - stock_data['Low']
    threshold = stock_data['daily_range'].mean() + 3 * stock_data['daily_range'].std()  # Setting a 3-sigma limit
    price_spikes = stock_data[stock_data['daily_range'] > threshold]
    if not price_spikes.empty:
     print("Unexpected Price Spikes Found:")
     print(price_spikes)
    else:
     print("No Unexpected Price Spikes Found.")


     # Close Column Smoothing
    stock_data['Close_Smoothed'] = stock_data['Close'].ewm(span=10, adjust=False).mean()

    # Droppinh unecessary columns
    stock_data.drop(columns = ['sma_50_check', 'ema_50_check', 'RSI_check', 'macd_line_check', 'ATR_check','TR' ],inplace = True)

    # Changing the type of Volume from float to int
    stock_data['Volume'] = stock_data['Volume'].astype(int)


    # Handling Missing Values
    from sklearn.impute import SimpleImputer
    def handle_missing(stock_data):
     if stock_data.isnull().sum().sum() > 0:
      impute = SimpleImputer(missing_values=np.nan,strategy='mean')
      impute.fit(stock_data[['Open','High','Low','Close','Volume','Dividends','sma_50','sma_200','ema_50','RSI','MACD','ATR','nifty_close','usd_inr','crude_oil']])
      stock_data[['Open','High','Low','Close','Volume','Dividends','sma_50','sma_200','ema_50','RSI','MACD','ATR','nifty_close','usd_inr','crude_oil']] = impute.transform(stock_data[['Open','High','Low','Close','Volume','Dividends','sma_50','sma_200','ema_50','RSI','MACD','ATR','nifty_close','usd_inr','crude_oil']])
     else:
      print("No missing values found")

    handle_missing(stock_data)

    stock_data = pd.DataFrame(stock_data)


     #removing duplicates
    def removing_duplicates(stock_data):
     if stock_data.duplicated().sum()>0:
      stock_data.drop_duplicates(inplace=True)
     else:
      print("No duplicates found")

    removing_duplicates(stock_data)

    duplicate_dates = stock_data.index.duplicated().sum()
    if duplicate_dates > 0:
     print(f"Duplicate Dates Found: {duplicate_dates}")
    else:
     print("No Duplicate Dates Found.")


    #  Data oveview
    stock_data.tail()
    stock_data.info()
    stock_data.describe()


    #  Store Cleaned Data in CSV File
    if not df.empty:
      if os.path.exists(stock_data):
       data.to_csv(stock_data, mode='a', header=False, index=False)  # Append new data
      else:
         data.to_csv(stock_data, index=False)  # Create a new file if not exists

    print("Daily cleaned stock data saved successfully.")

  except Exception as e:
      print(f"Error: {e}")

